In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import os

os.environ["HOPSWORKS_API_KEY"] = "qf4YnRgKPjj8IeMQ.R4Xeeq4Ge0CVPmW1qLCSOBJLCyqJ9XQoEg3jFnn9WmfPXqO6p5tWvHIGSh1DKnr8"
os.environ["HOPSWORKS_PROJECT_NAME"] = "new_york_taxi"

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# print("API Key:", api_key)
print("Project Name:", project_name)

# Initialize connection to Hopsworks  
project = hopsworks.login(api_key_value=api_key, project=project_name)
print(f"Successfully connected to Hopsworks project: {project_name}")

Project Name: new_york_taxi
2025-03-03 10:03:11,670 INFO: Initializing external client
2025-03-03 10:03:11,671 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 10:03:12,711 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Successfully connected to Hopsworks project: new_york_taxi


In [4]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [5]:
df = fetch_hourly_rides(12)

2025-03-03 10:03:13,760 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 10:03:13,763 INFO: Initializing external client
2025-03-03 10:03:13,764 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 10:03:14,580 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.89s) 


In [6]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-03 10:00:00+00:00,72,2
1,2025-03-03 03:00:00+00:00,50,1
2,2025-03-03 00:00:00+00:00,137,14
3,2025-03-03 13:00:00+00:00,212,0
4,2025-03-03 00:00:00+00:00,131,0
...,...,...,...
4011,2025-03-03 00:00:00+00:00,160,0
4012,2025-03-03 13:00:00+00:00,67,0
4013,2025-03-03 03:00:00+00:00,197,1
4014,2025-03-03 00:00:00+00:00,91,0


In [7]:
df_pred = fetch_predictions(240)

2025-03-03 10:03:26,888 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 10:03:26,893 INFO: Initializing external client
2025-03-03 10:03:26,894 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 10:03:28,094 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (30.97s) 


In [8]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,177,1.0,2025-03-03 15:00:00+00:00
1,72,2.0,2025-03-03 15:00:00+00:00
2,16,0.0,2025-03-03 15:00:00+00:00
3,209,4.0,2025-03-03 15:00:00+00:00
4,68,123.0,2025-03-03 15:00:00+00:00
...,...,...,...
246,139,0.0,2025-03-03 15:00:00+00:00
247,13,29.0,2025-03-03 15:00:00+00:00
248,206,0.0,2025-03-03 15:00:00+00:00
249,167,1.0,2025-03-03 15:00:00+00:00


In [9]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [10]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand


In [11]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [12]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [13]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [14]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [15]:
mae_by_hour["MAE"].mean()

nan